In [1]:
from Data_Enrichment import get_features

RAW_DIR = "data/raw"
df_feats, feature_cols = get_features(RAW_DIR)
df_feats = df_feats[(df_feats['season_end_year'] != 2025) & (df_feats['season_end_year'] != 2026)]

/workspaces/Machine-learning/Data_Enrichment.py:232: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["ballon_dor_winner"] = df["ballon_dor_winner"].fillna(False).astype(bool)


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = df_feats[[
    'goals',
    'assists',
    'penalty_goals',
    'yellow_cards',
    'direct_red_cards',
    'minutes_played',
    'height',
    'g_per90',
    'a_per90',
    'pen_share',
    'discipline_rate',
    'clean_sheet_rate'
]]



In [4]:
a = df.select_dtypes(include=np.number)
a.head()

,goals,assists,penalty_goals,yellow_cards,direct_red_cards,minutes_played,height,g_per90,a_per90,pen_share,discipline_rate,clean_sheet_rate
0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0,0,0,0,0.0,195.0,0.0,0.0,0.0,0.0,0.0
8,0.0,1,0,3,0,0.0,187.0,0.0,0.0,0.0,0.5,0.0
9,0.0,1,0,1,0,0.0,187.0,0.0,0.0,0.0,0.5,0.0


In [5]:
from sklearn.preprocessing import StandardScaler
# Standardize the features
scaler = StandardScaler() #Z score
X1 = scaler.fit_transform(a)

In [ ]:
print(X1)

In [6]:
from sklearn.cluster import KMeans
# We apply it to our date
kmeans = KMeans(n_clusters=8, random_state=1,n_init=50).fit(X1)

# Here it is the clustering result
labels = kmeans.labels_
print(labels)
a.dtypes


[5 5 0 ... 0 4 5]


goals               float64
assists               int64
penalty_goals         int64
yellow_cards          int64
direct_red_cards      int64
minutes_played      float64
height              float64
g_per90             float64
a_per90             float64
pen_share           float64
discipline_rate     float64
clean_sheet_rate    float64
dtype: object

In [7]:
selected_features = [
    'goals',
    'assists',
    'penalty_goals',
    'yellow_cards',
    'direct_red_cards',
    'minutes_played',
    'height',
    'g_per90',
    'a_per90',
    'pen_share',
    'discipline_rate',
    'clean_sheet_rate'
]


In [8]:
df_feats['kmeans_cluster'] = labels.astype(str)  # convertir a string opcional

# Ahora sí podemos agrupar
cluster_means = df_feats.groupby('kmeans_cluster')[selected_features].mean().T
print(cluster_means)

kmeans_cluster             0           1            2           3           4  \
goals               0.385949   23.799127     0.000000    8.053781    1.373263   
assists             0.342474    8.729258     0.080443    6.111289    1.516263   
penalty_goals       0.004191    4.056769     0.000000    0.322151    0.010603   
yellow_cards        0.467926    4.441048     1.031734    3.517572    4.843531   
direct_red_cards    0.000000    0.078603     0.000000    0.020767    0.000000   
minutes_played    110.028699  191.388646  2100.940959  410.550053  818.944261   
height            182.841870  181.681223   190.344649  180.181576  182.438888   
g_per90             0.128787   12.816952     0.000000    2.670582    0.211063   
a_per90             0.037936    4.352201     0.003201    1.543513    0.120097   
pen_share           0.000920    0.184807     0.000000    0.036452    0.002295   
discipline_rate     0.034445    0.133063     0.044498    0.113019    0.249810   
clean_sheet_rate    0.001325

In [ ]:
df_feats.groupby('kmeans_cluster')[selected_features].mean().T


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Seleccionamos solo las features que queremos analizar + cluster
df_plot = df_feats[selected_features + ['kmeans_cluster']]

# Calculamos la media de cada feature por cluster
cluster_means = df_plot.groupby('kmeans_cluster')[selected_features].mean().T

# Heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(cluster_means, annot=True, cmap='viridis', fmt=".2f")
plt.title('Media de features por cluster')
plt.xlabel('Cluster')
plt.ylabel('Feature')
plt.show()


In [ ]:
# In KMeans function, the field inertia_ is the percentage of variance explained by the number of clusters
wcss = []
K = range(1, 15)

for k in K:
    kmeans = KMeans(n_clusters=k, init='k-means++', n_init=10, random_state=42)
    kmeans.fit(X1)
    wcss.append(kmeans.inertia_)

# Plot Elbow
plt.figure(figsize=(8, 5))
plt.plot(K, wcss, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS (Inertia)')
plt.title('Elbow Method')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
# ---------------------------------------
# 2. Silhouette Method
# ---------------------------------------

silhouette_scores = []
K = range(2, 11)  # silhouette requires at least 2 clusters

for k in K:
    kmeans = KMeans(n_clusters=k, init='k-means++', n_init=10, random_state=42)
    labels = kmeans.fit_predict(X1)
    score = silhouette_score(X1, labels)
    silhouette_scores.append(score)

# Plot Silhouette Scores
plt.figure(figsize=(8, 5))
plt.plot(K, silhouette_scores, marker='s', color='green')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Method')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

# ---------------------------------------
# 3. Define Gap Statistic function
# ---------------------------------------
def compute_wcss(X, n_clusters):
    """Compute within-cluster sum of squares (WCSS) for given k"""
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    kmeans.fit(X)
    return kmeans.inertia_

def gap_statistic(X, max_k=10, B=10):
    """Compute gap statistic for k = 1 to max_k with B reference datasets"""
    n_samples, n_features = X.shape
    wcss = np.zeros(max_k)
    wcss_ref = np.zeros((B, max_k))

    # Bounds for uniform reference distribution
    xmin = X.min(axis=0)
    xmax = X.max(axis=0)

    for k in range(1, max_k + 1):
        # Real WCSS
        wcss[k - 1] = compute_wcss(X, k)

        # Reference WCSS
        for b in range(B):
            X_ref = np.random.uniform(xmin, xmax, size=(n_samples, n_features))
            wcss_ref[b, k - 1] = compute_wcss(X_ref, k)

    # Gap = log(E[WCSS_ref]) - log(WCSS_data)
    log_wcss = np.log(wcss)
    log_wcss_ref = np.log(wcss_ref)
    gap = log_wcss_ref.mean(axis=0) - log_wcss
    gap_std = log_wcss_ref.std(axis=0) * np.sqrt(1 + 1 / B)

    return gap, gap_std, wcss, wcss_ref

# ---------------------------------------
# 3. Run and plot the gap statistic
# ---------------------------------------
max_k = 10
gap, gap_std, wcss, wcss_ref = gap_statistic(X1, max_k=max_k, B=10)

# Optimal k (according to "1-SE" rule)
optimal_k = np.argmax(gap[:-1] >= gap[1:] - gap_std[1:]) + 1

# Plot
plt.figure(figsize=(8, 5))
plt.plot(range(1, max_k + 1), gap, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Gap Value')
plt.title('Gap Statistic')
plt.grid(True)
plt.axvline(optimal_k, color='red', linestyle='--', label=f'Optimal k = {optimal_k}')
plt.legend()
plt.show()

# Print optimal k
print(f"Suggested optimal number of clusters (Gap Statistic): {optimal_k}")
